# init

In [1]:
from pprint import pprint as pp
import random
import threading
import time


In [2]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(threadName)s: %(message)s')

In [3]:
import queue
import concurrent.futures

# demo

### demo 001

In [11]:

def do_task():
    for _ in range(300):
        # logging.debug('print B')
        print('B', end='')
        # time.sleep(1)


th = threading.Thread(target=do_task)
th.start()

for _ in range(300):
    # logging.debug('print A')
    print('A', end='')
    # time.sleep(1)

th.join()
print('hello world!') # this line is not executed until th.join() is finished


BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAhello world!


In [8]:
print('hello world')

hello world


### demo 03

In [12]:
'''
PASSING ARGUMENTS
Version A: Using the Thread's constructor
'''

import threading


def do_task(a: int, b: float, c: str):
    print(f'{a}  {b}  {c}')


th_foo = threading.Thread(target=do_task, args=(1, 2, 'red'))
th_bar = threading.Thread(target=do_task, args=(3, 4, 'blue'))

th_foo.start()
th_bar.start()


1  2  red
3  4  blue


In [13]:
'''
PASSING ARGUMENTS
Version B: Using lambdas
'''

import threading


def do_task(a: int, b: float, c: str):
    print(f'{a}  {b}  {c}')


th_foo = threading.Thread(target=lambda: do_task(1, 2, 'red'))
th_bar = threading.Thread(target=lambda: do_task(3, 4, 'blue'))

th_foo.start()
th_bar.start()


1  2  red
3  4  blue


### list_threads

In [19]:
def do_task(index):
    time.sleep(0.5)
    print(index, end='')



NUM_THREADS = 5
tasks = []

for i in range(NUM_THREADS):
    arg = i
    tasks.append(threading.Thread( target= do_task, args=(i,)))

for th in tasks:
    th.start()

02134

### terminate a thread

In [20]:
'''
FORCING A THREAD TO TERMINATE (i.e. killing the thread)
Using a flag to notify the thread
'''

flag_stop = False


def do_task():
    while True:
        if flag_stop:
            break

        print('Running...')
        time.sleep(1)


th = threading.Thread(target=do_task)
th.start()

time.sleep(3)
flag_stop = True


Running...
Running...
Running...


### return value(s)

In [22]:
def double_value(value):
    return value * 2

res = {}

th_foo = threading.Thread( target=lambda: res.update({ 'foo': double_value(5) }) )
th_bar = threading.Thread( target=lambda: res.update({ 'bar': double_value(80) }) )

th_foo.start()
th_bar.start()

# Wait until th_foo and th_far finish
th_foo.join()
th_bar.join()

pp(res)

{'bar': 160, 'foo': 10}


### .join() --- a synchronization mechanism

In [56]:
def double_value(value):
    time.sleep(random.randint(0, 1))
    return value * 2

running = True

def taskMgt():
    while running:
        res = {}
        th1 = threading.Thread( target=lambda: res.update({ 'foo': double_value(5) }) )
        th2 = threading.Thread( target=lambda: res.update({ 'bar': double_value(80) }) ) 
        th1.start()
        th2.start()
        th1.join()
        th2.join()
        pp(res)
        time.sleep(0.5)

# Wait until th_foo and th_far finish
# th_foo.join()
# th_bar.join()
th3 = threading.Thread( target=taskMgt )
th3.start()

{'bar': 160, 'foo': 10}
{'bar': 160, 'foo': 10}
{'bar': 160, 'foo': 10}
{'bar': 160, 'foo': 10}
{'bar': 160, 'foo': 10}


In [57]:
running = False

{'bar': 160, 'foo': 10}


### detaching

In [4]:
def do_task():
    print('foo is starting...')
    time.sleep(2)
    print('foo is exiting...')



# th_foo = threading.Thread(target=do_task)
th_foo = threading.Thread(target=do_task, daemon=True) # will run continuously
th_foo.start()

# If I comment this statement,
# th_foo will be forced into terminating with main thread
time.sleep(3)

print('Main thread is exiting')

foo is starting...
foo is exiting...
Main thread is exiting


### a exec service and thread pools

In [7]:
'''
EXECUTOR SERVICES AND THREAD POOLS
Version A: The executor service (of which thread pool) containing a single thread
'''

from concurrent.futures import ThreadPoolExecutor


def do_task():
    print('Hello the Executor Service')


executor = ThreadPoolExecutor(max_workers=1)

executor.submit(lambda: print('Hello World'))
executor.submit(do_task)
executor.submit(lambda: print('Hello World~~~'))

executor.shutdown(wait=True)


Hello World
Hello the Executor Service
Hello World~~~


In [3]:
'''
EXECUTOR SERVICES AND THREAD POOLS
Version B: The executor service containing multiple threads
'''

from concurrent.futures import ThreadPoolExecutor


def do_task(name: str):
    print(f'Task {name} is starting')
    time.sleep(3)
    print(f'Task {name} is completed')


NUM_TASKS = 5
executor = ThreadPoolExecutor(max_workers=2)

for i in range(NUM_TASKS):
    task_name = chr(i + 65)
    executor.submit(do_task, task_name) # push the task into the thread pool

executor.shutdown(wait=True) # wait until all tasks are completed, then exit


Task A is starting
Task B is starting
Task B is completed
Task C is starting
Task A is completed
Task D is starting
Task C is completed
Task E is starting
Task D is completed
Task E is completed


Version C01: The executor service and Future objects\
`executor.submit(callback, *args, **kwargs) -> Future object representing the execution of the task`

In [5]:

def get_squared(x):
    return x * x


executor = ThreadPoolExecutor(max_workers=1)

future = executor.submit(get_squared, 7) 
# print(future.done())

print(future.result())

executor.shutdown(wait=True)


True
49


In [6]:
def get_squared(x):
    time.sleep(3)
    return x * x


executor = ThreadPoolExecutor(max_workers=1)

future = executor.submit(get_squared, 9)

print('Calculating...')
print(future.result())

executor.shutdown(wait=True)


Calculating...
81


## race condition

### race condition without data race

In [8]:
def do_task(index: int):
    time.sleep(1)
    print(index, end='')


NUM_THREADS = 4
lstth = []

for i in range(NUM_THREADS):
    lstth.append(threading.Thread(target=do_task, args=(i,)))

for th in lstth:
    th.start()


0132

In [9]:
'''
DATA RACES
Version 01: Without multithreading
'''


def get_result(n: int):
    a = [False] * (n + 1)

    for i in range(1, n + 1):
        if i % 2 == 0 or i % 3 == 0:
            a[i] = True

    res = a.count(True)
    return res


N = 8
result = get_result(N)
print('Number of integers that are divisible by 2 or 3 is:', result)


Number of integers that are divisible by 2 or 3 is: 5


In [12]:
'''
DATA RACES
Version 02: Multithreading
'''

import threading


def count_div_2(a: list, n: int):
    for i in range(2, n + 1, 2):
        a[i] = True


def count_div_3(a: list, n: int):
    for i in range(3, n + 1, 3):
        a[i] = True


N = 8
A = [False] * (N + 1)

th_div_2 = threading.Thread(target=count_div_2, args=(A, N))
th_div_3 = threading.Thread(target=count_div_3, args=(A, N))

th_div_2.start()
th_div_3.start()
th_div_2.join()
th_div_3.join()

result = A.count(True)

print('Number of integers that are divisible by 2 or 3 is:', result)


Number of integers that are divisible by 2 or 3 is: 5


### race condition with data race

In [19]:
counter = 0

def do_task():
    global counter

    for _ in range(1000):
        temp = counter + 1
        time.sleep(0.0001)
        counter = temp



lstth = [threading.Thread(target=do_task) for _ in range(10)]

for th in lstth:
    th.start()

for th in lstth:
    th.join()

print('counter =', counter)
# We are NOT sure that counter = 32000

counter = 1000


In [25]:
counter = 0


def do_task_a():
    global counter
    time.sleep(random.randint(0, 1))

    while counter < 10:
        counter += 1

    print('A won !!!')


def do_task_b():
    global counter
    time.sleep(random.randint(0, 1))

    while counter > -10:
        counter -= 1

    print('B won !!!')


threading.Thread(target=do_task_a).start()
threading.Thread(target=do_task_b).start()


A won !!!


B won !!!


# Avoid race condition

## Synchronization primitive

### mutex by threading.Lock()

In [27]:
'''
MUTEXES
In Python, Lock objects can be used as mutexes
'''
mutex = threading.Lock()
counter = 0

def do_task():
    global counter

    mutex.acquire()

    for _ in range(1000):
        temp = counter + 1
        time.sleep(0.0001)
        counter = temp

    mutex.release()


NUM_THREADS = 10
lstth = [threading.Thread(target=do_task) for _ in range(NUM_THREADS)]

for th in lstth:
    th.start()

for th in lstth:
    th.join()

print('counter =', counter)
# We are sure that counter = 32000


counter = 10000


In [29]:
'''
SYNCHRONIZED BLOCKS
'''

mutex = threading.Lock()
counter = 0


def do_task():
    global counter

    # synchronized block
    with mutex: # with statement will automatically acquire and  release the lock
        for _ in range(1000):
            temp = counter + 1
            time.sleep(0.0001)
            counter = temp


NUM_THREADS = 6
lstth = [threading.Thread(target=do_task) for _ in range(NUM_THREADS)]

for th in lstth:
    th.start()

for th in lstth:
    th.join()

print('counter =', counter)
# We are sure that counter = 32000


counter = 6000


### Monitors _Concurrency && OOP_

A `monitor` is a thread-safe class, object, or module that wraps around a `mutex` \
in order to safely allow access to a method or variable by more than one thread.

In [31]:
'''
MONITORS
Implementation of a monitor for managing a counter
'''
class Monitor:
    def __init__(self, res: dict, field_name: str):
        self.__lock = threading.Lock()
        self.__res = res
        self.__field_name = field_name

    def increase_counter(self):
        with self.__lock:
            tmp = self.__res[self.__field_name] + 1
            time.sleep(0.0001)
            self.__res[self.__field_name] = tmp


def do_task(mon: Monitor):
    for _ in range(1000):
        mon.increase_counter()


In [32]:
result = {'data': 0}
monitor = Monitor(result, 'data')

NUM_THREADS = 32
lstth = [threading.Thread(target=do_task, args=(monitor,))
         for _ in range(NUM_THREADS)]

for th in lstth:
    th.start()

for th in lstth:
    th.join()

print('counter =', result['data'])
# We are sure that counter = 32000


counter = 32000


### reentrant lock

problem

In [ ]:
lock = threading.Lock()


def do_task():
    with lock:
        print('First time acquiring the resource')
        with lock:
            print('Second time acquiring the resource')


th = threading.Thread(target=do_task)
th.start()

# The thread th shall meet deadlock.
# So, you will never get output "Second time the acquiring resource".


solution

In [33]:
lock = threading.RLock() # reentrant lock


def do_task():
    with lock:
        print('First time acquiring the resource')
        with lock:
            print('Second time acquiring the resource')


th = threading.Thread(target=do_task)
th.start()
th.join()


First time acquiring the resource
Second time acquiring the resource


In [ ]:
lock = threading.RLock()


def do_task():
    time.sleep(1)
    with lock:
        logging.debug('level-0 enter')
        with lock:
            logging.debug('level-1 enter')
    logging.debug('exit')


NUM_THREADS = 3

for i in range(NUM_THREADS):
    threading.Thread(target=do_task, name=chr(i+ 65)).start()


A: level-0 enter
A: level-1 enter
A: exit
C: level-0 enter
C: level-1 enter
C: exit
B: level-0 enter
B: level-1 enter
B: exit


### barriers and latches

### read-write lock

### condition variable

In [9]:
symbols = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
timeFrame = ['15m', '1h']

keys = [(symbol, timeframe) for symbol in symbols for timeframe in timeFrame]
conditions = {key: threading.Condition() for key in keys}

def foo(key: tuple):
    logging.debug('foo waiting')
    with conditions[key]:
        conditions[key].wait()
    print('foo resumed')


def bar(key: tuple):
    time.sleep(3)
    logging.debug('bar notify')
    with conditions[key]:
        conditions[key].notify()

tasks_foo = [threading.Thread(target=foo, args=(key,)) for key in keys]
tasks_bar = [threading.Thread(target=bar, args=(key,)) for key in keys]

for th in tasks_foo:
    th.start()
for th in tasks_bar:
    th.start()



Thread-123 (foo): foo waiting
Thread-124 (foo): foo waiting
Thread-125 (foo): foo waiting
Thread-126 (foo): foo waiting
Thread-127 (foo): foo waiting
Thread-128 (foo): foo waiting
Thread-129 (foo): foo waiting
Thread-130 (foo): foo waiting
Thread-131 (foo): foo waiting
Thread-132 (foo): foo waiting
Thread-133 (foo): foo waiting
Thread-134 (foo): foo waiting
Thread-135 (foo): foo waiting
Thread-136 (foo): foo waiting
Thread-137 (foo): foo waiting
Thread-138 (foo): foo waiting
Thread-139 (foo): foo waiting
Thread-140 (foo): foo waiting
Thread-141 (foo): foo waiting
Thread-142 (foo): foo waiting
Thread-161 (bar): bar notify
Thread-155 (bar): bar notify
Thread-154 (bar): bar notify
Thread-149 (bar): bar notify
Thread-150 (bar): bar notify
Thread-148 (bar): bar notify
Thread-158 (bar): bar notify
Thread-159 (bar): bar notify
Thread-144 (bar): bar notify
Thread-157 (bar): bar notify
Thread-143 (bar): bar notify
Thread-153 (bar): bar notify
Thread-152 (bar): bar notify
Thread-151 (bar): bar n

foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
foo resumed
main thread finished


In [10]:
"""
version 2: 
reuse threads to handle the same tasks continuously
by maintain a thread pool 
"""
symbols = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
timeFrame = ['15m', '1h']

keys = [(symbol, timeframe) for symbol in symbols for timeframe in timeFrame]
conditions = {key: threading.Condition() for key in keys}


def foo(key: tuple):
    logging.debug('foo waiting')
    with conditions[key]:
        conditions[key].wait()
    print(f'foo @{key} resumed')


def bar(key: tuple):
    time.sleep(3)
    logging.debug(f'bar notify {key}')
    with conditions[key]:
        conditions[key].notify()

thread_pool = {key: threading.Thread(target=foo, args=(key,)) for key in keys}
for th in thread_pool.values():
    th.start()

# To reuse the same threads:
for key in keys:
    bar(key)


for th in tasks_foo:
    th.join()
print('main thread finished')
    

Thread-163 (foo): foo waiting
Thread-164 (foo): foo waiting
Thread-165 (foo): foo waiting
Thread-166 (foo): foo waiting
Thread-167 (foo): foo waiting
Thread-168 (foo): foo waiting
Thread-169 (foo): foo waiting
Thread-170 (foo): foo waiting
Thread-171 (foo): foo waiting
Thread-172 (foo): foo waiting
Thread-173 (foo): foo waiting
Thread-174 (foo): foo waiting
Thread-175 (foo): foo waiting
Thread-176 (foo): foo waiting
Thread-177 (foo): foo waiting
Thread-178 (foo): foo waiting
Thread-179 (foo): foo waiting
Thread-180 (foo): foo waiting
Thread-181 (foo): foo waiting
Thread-182 (foo): foo waiting
MainThread: bar notify ('A', '15m')


foo @('A', '15m') resumed


MainThread: bar notify ('A', '1h')


foo @('A', '1h') resumed


MainThread: bar notify ('B', '15m')


...

foo @('I', '1h') resumed


MainThread: bar notify ('J', '15m')


foo @('J', '15m') resumed


MainThread: bar notify ('J', '1h')


main thread finishedfoo @('J', '1h') resumed



In [6]:
"""
version 3:
use concurrent.futures module to create a thread pool
"""
import concurrent.futures

symbols = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
timeFrame = ['15m', '1h']

keys = [(symbol, timeframe) for symbol in symbols for timeframe in timeFrame]
conditions = {key: threading.Condition() for key in keys}
get_signals = {key: threading.Condition() for key in keys}

def foo(key: tuple):
    logging.debug(f'foo @{key} waiting')
    with conditions[key]:
        conditions[key].wait()
        with get_signals[key]:
            get_signals[key].wait()
    print(f'foo @{key} resumed')
        


# def bar(key: tuple):
#     time.sleep(3)
#     logging.debug('bar notify @{key}')
#     with conditions[key]:
#         conditions[key].notify()


with concurrent.futures.ThreadPoolExecutor(max_workers=len(keys)) as executor:
    # Submit tasks to the thread pool
    for key in keys:
        executor.submit(foo, key)

class Notice:
    def __init__(self, keys: list[tuple], conditions, get_signals):
        self.keys = keys
        self.conditions = conditions
        self.get_signals = get_signals
    
    def startWatching(self):
        for keys in self.keys:
            logging.debug(f'{key} start watching')
            time.sleep(random.randint(1, 3))
            with self.conditions[keys]:
                self.conditions[keys].notify()
    def emit_signal(self):
        for keys in self.keys:
            time.sleep(random.randint(2, 8))
            with self.get_signals[keys]:
                self.get_signals[keys].notify()

notice = Notice(keys, conditions, get_signals)
notice.startWatching()
notice.emit_signal()
    


    




ThreadPoolExecutor-2_0: foo @('A', '15m') waiting
ThreadPoolExecutor-2_1: foo @('A', '1h') waiting
ThreadPoolExecutor-2_2: foo @('B', '15m') waiting
ThreadPoolExecutor-2_3: foo @('B', '1h') waiting
ThreadPoolExecutor-2_4: foo @('C', '15m') waiting
ThreadPoolExecutor-2_5: foo @('C', '1h') waiting
ThreadPoolExecutor-2_6: foo @('D', '15m') waiting
ThreadPoolExecutor-2_7: foo @('D', '1h') waiting
ThreadPoolExecutor-2_8: foo @('E', '15m') waiting
ThreadPoolExecutor-2_9: foo @('E', '1h') waiting
ThreadPoolExecutor-2_10: foo @('F', '15m') waiting
ThreadPoolExecutor-2_11: foo @('F', '1h') waiting
ThreadPoolExecutor-2_12: foo @('G', '15m') waiting
ThreadPoolExecutor-2_13: foo @('G', '1h') waiting
ThreadPoolExecutor-2_14: foo @('H', '15m') waiting
ThreadPoolExecutor-2_15: foo @('H', '1h') waiting
ThreadPoolExecutor-2_16: foo @('I', '15m') waiting
ThreadPoolExecutor-2_17: foo @('I', '1h') waiting
ThreadPoolExecutor-2_18: foo @('J', '15m') waiting
ThreadPoolExecutor-2_19: foo @('J', '1h') waiting


### block queues

In [ ]:
import queue


def worker(q: queue.Queue):
    while True:
        item = q.get()
        if item is None:
            break # exit the thread
        function, args = item
        function(*args)


q = queue.Queue()
t = threading.Thread(target=worker, args=(q,))
t.start()


def my_function(arg1, arg2):
    print(arg1, arg2)


q.put((my_function, ("Hello", "world")))

q.put(None)
t.join()


In [32]:
from concurrent.futures import thread


symbols = ['A', 'B', 'C', ]
timeFrame = ['15m', '1h']
keys = [(symbol, timeframe) for symbol in symbols for timeframe in timeFrame]

# triggers = { key: threading.Event() for key in keys} 

def worker(task_queue: queue.Queue):
    logging.debug(f'{threading.current_thread().daemon}')
    with concurrent.futures.ThreadPoolExecutor() as executor:
        while True:
            task = task_queue.get()
            executor.submit(handle_task, task)

def handle_task(task):
    logging.debug(f'{threading.current_thread().daemon})')
    logging.debug(f'handle task {task}')
    time.sleep(random.randit(1, 3))

def add_task(task_queue: queue.Queue, task):
    time.sleep(5) # wait for 5 seconds
    task_queue.put(task) # add new task
    logging.debug(f'added new task: {task}')

task_queue = queue.Queue()
t1 = threading.Thread(target=worker, args=(task_queue,))
def configure_tasks():
    for key in keys:
        # triggers[key].set()
        add_task(task_queue, key)
t2 = threading.Thread(target=configure_tasks,)

t1.start()
t2.start()

print('non-blocking main thread')

for i in range(1, 3):
    time.sleep(random.randint(1, 3))
# check if the thread is still alive
add_task(task_queue, 'new task')
print('main thread finished, but the worker thread is still alive')

Thread-38 (worker): False


non-blocking main thread


Thread-39 (configure_tasks): added new task: ('A', '15m')
ThreadPoolExecutor-18_0: False)
ThreadPoolExecutor-18_0: handle task ('A', '15m')
Thread-39 (configure_tasks): added new task: ('A', '1h')
ThreadPoolExecutor-18_0: False)
ThreadPoolExecutor-18_0: handle task ('A', '1h')
MainThread: added new task: new task


main thread finished, but the worker thread is still alive


ThreadPoolExecutor-18_0: False)
ThreadPoolExecutor-18_0: handle task new task


In [28]:
print('hello, world~')

hello, world~


Thread-31 (configure_tasks): added new task: ('B', '15m')
ThreadPoolExecutor-14_1: handle task ('B', '15m')
Thread-31 (configure_tasks): added new task: ('B', '1h')
ThreadPoolExecutor-14_0: handle task ('B', '1h')
Thread-31 (configure_tasks): added new task: ('C', '15m')
ThreadPoolExecutor-14_1: handle task ('C', '15m')
Thread-31 (configure_tasks): added new task: ('C', '1h')
ThreadPoolExecutor-14_0: handle task ('C', '1h')


### thread-local storage

## non-blocking methods

### demo 01

In [8]:
def mytest(task_queue: queue.Queue):
    logging.debug('now get the task from the queue')
    while True:
        print(task_queue.get())


In [9]:
task_queue = queue.Queue()
task_queue.put(0)
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.submit(mytest, task_queue)
logging.debug('blocking main thread? Yes, it is.')
task_queue.put(1)
task_queue.put(2)


ThreadPoolExecutor-0_0: now get the task from the queue


0


### Next level play: nested threads

In [4]:
def worker(task_queue: queue.Queue):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        logging.debug('now get the task from the queue')
        while True:
            task = task_queue.get()
            # if task is None:
            #     break
            executor.submit(handle_task, task)

def handle_task(task):
    logging.debug(f'handling task {task}')
    # Do some work...
    print(task)

task_queue = queue.Queue()
task_queue.put(0)
t1 = threading.Thread(target=worker, args=(task_queue,))
t1.start()
task_queue.put(1)
task_queue.put(2)
# t1.join() # if you want to wait for the worker thread to finish
logging.debug('blocking main thread? No, it isn\'t.')

Thread-3 (worker): now get the task from the queue
MainThread: blocking main thread? No, it isn't.
ThreadPoolExecutor-0_0: handling task 0
ThreadPoolExecutor-0_1: handling task 1
ThreadPoolExecutor-0_2: handling task 2


01

2


### solution A --  nested threads with executor running at background

In [4]:
def worker(task_queue: queue.Queue, ) :
    with concurrent.futures.ThreadPoolExecutor() as executor:
        logging.debug('now get the task from the queue')
        while True:
            task = task_queue.get()
            executor.submit(handle_task, task)

def handle_task(task):
    logging.debug(f'handling task {task}')
    # Do some work...
    print(task)
    time.sleep(1, 10)

task_queue = queue.Queue()
task_queue.put(0)
t1 = threading.Thread(target=worker, args=(task_queue,), daemon=True)
t1.start()
task_queue.put(1)
task_queue.put(2)
task_queue.put(3)

# The worker thread will run in the background
# The main thread will not be blocked
logging.debug('blocking main thread? No, it isn\'t')
def main():
    for i in range(3):
        time.sleep(random.randint(0, 3))
        task_queue.put(random.randint(0, 101))
        logging.debug('main thread is running')
    logging.debug('main thread finished')
main()



Thread-3 (worker): now get the task from the queue
MainThread: blocking main thread? No, it isn't
ThreadPoolExecutor-0_0: handling task 0
ThreadPoolExecutor-0_1: handling task 1
ThreadPoolExecutor-0_2: handling task 2
ThreadPoolExecutor-0_0: handling task 3


0
1
2
3


MainThread: main thread is running
ThreadPoolExecutor-0_1: handling task 59


59


MainThread: main thread is running
ThreadPoolExecutor-0_0: handling task 41
MainThread: main thread is running
ThreadPoolExecutor-0_2: handling task 51
MainThread: main thread finished


41
51


### solutinon B -- simple threading.Thread only suit for single task

In [5]:
def mytest(task_queue: queue.Queue):
    logging.debug('now get the task from the queue>>>')
    while True:
        task = task_queue.get()
        handle_task(task)

def handle_task(task):
    logging.debug(f'handling task {task}')
    # do sth
    print(task)
    time.sleep(random.randint(1,10))

task_queue = queue.Queue()
task_queue.put(0)
t1 = threading.Thread(target=mytest, args=(task_queue,), daemon=True )
t1.start()
task_queue.put(1)
task_queue.put(2)
task_queue.put(3)
print('blocking main thread? No, it is not. ')

def main():
    for i in range(3):
        time.sleep(random.randint(1, 3))
        task_queue.put(random.randint(0, 101))
        logging.debug('main thread is still running')
    logging.debug('main thread finished')
main()

Thread-4 (mytest): now get the task from the queue>>>
Thread-4 (mytest): handling task 0


blocking main thread? No, it is not. 
0


MainThread: main thread is still running
Thread-4 (mytest): handling task 1


1


MainThread: main thread is still running
Thread-4 (mytest): handling task 2


2


MainThread: main thread is still running
MainThread: main thread finished


Thread-4 (mytest): handling task 3


3
